In [5]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

import pandas as pd
import numpy as np

In [6]:
with create_engine('sqlite:///ulam.db').connect() as conn:
    df_ing = pd.read_sql("""
    SELECT * FROM rekado
    """, conn)
    
display(df_ing)

,dish_name,yeast,wrapper,worcestershire_sauce,winged_bean,vinegar,turmeric,tomato,tofu,toasted_rice_powder,...,pasta,shrimp,pineapple,water,salt,rice,corn,chicken,beef,banana
0,Sinarsahang Manok,0.0,0.0,0.0,0.0,0.0000,0.0,2.5,0.0,0.0,...,0.0,0.0,0.0,1.50,0.000000,0.0,0.000,680.389364,0.000000,0.0
1,Pinoy Chicken Curry Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.00,0.000000,0.0,0.000,907.185819,0.000000,0.0
2,Ketchup Fried Chicken,0.0,0.0,0.0,0.0,0.3125,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.000000,0.0,0.000,3.000000,0.000000,0.0
3,Chicken Barbecue,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,2.000000,0.0,0.000,4.000000,0.000000,0.0
4,Chicken Paksiw Recipe,0.0,0.0,0.0,0.0,0.5000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.00,1.000000,0.0,0.000,453.592909,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,Papaitan Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.00,0.156250,0.0,0.000,0.000000,226.796455,0.0
278,Morcon Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00,0.005208,0.0,0.000,0.000000,907.185819,0.0
279,Beef Mechado Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.00,1.000000,0.0,0.000,0.000000,907.185819,0.0
280,Bulalo Recipe,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.25,0.000000,0.0,1.125,0.000000,907.185819,0.0


In [7]:
list(df_ing.columns)

['dish_name',
 'yeast',
 'wrapper',
 'worcestershire_sauce',
 'winged_bean',
 'vinegar',
 'turmeric',
 'tomato',
 'tofu',
 'toasted_rice_powder',
 'taro',
 'sweet_potato',
 'sugar',
 'star_anise',
 'squash',
 'soy_sauce',
 'sinigang_mix',
 'shortening',
 'sesame_oil',
 'scallion',
 'sayote',
 'salted_egg',
 'safflower_oil',
 'raisins',
 'radish',
 'potato',
 'pork_stock',
 'pork_insides',
 'pork_fat',
 'pork_cube',
 'pork_broth',
 'pork_blood',
 'pork_and_beans',
 'pork',
 'pie_crust',
 'pickle',
 'peppercorn',
 'pepper_leaf',
 'pechay',
 'peanut_butter',
 'peanut',
 'patola',
 'parsley',
 'paprika',
 'papaya',
 'oyster_sauce',
 'onion',
 'olive_oil',
 'olive',
 'okra',
 'nutmeg',
 'noodle',
 'mushroom',
 'munggo',
 'mirin',
 'mayonnaise',
 'marinade',
 'malunggay',
 'liver_spread',
 'liquid_seasoning',
 'lemongrass',
 'leeks',
 'lechon_sauce',
 'kasubha',
 'kangkong',
 'jicama',
 'jackfruit',
 'ice',
 'hotdog',
 'hot_sauce',
 'honey',
 'hoisin_sauce',
 'green_pea',
 'green_bean',
 'gl